# 한국어 방언 탐지 모델 학습

이 노트북은 Wav2Vec2 모델을 한국어 방언 분류에 맞게 파인튜닝합니다.

## 필요한 사전 준비

### 1. 데이터셋 구조
SSD에 저장된 데이터를 다음과 같은 구조로 준비해주세요:
```
/path/to/your/dialect_dataset/
├── standard/          # 표준어 (서울/수도권)
│   ├── audio_001.wav
│   ├── audio_002.wav
│   └── ...
├── gyeongsang/        # 경상도 방언
│   ├── audio_001.wav
│   └── ...
├── jeolla/            # 전라도 방언
│   ├── audio_001.wav
│   └── ...
├── chungcheong/       # 충청도 방언
│   ├── audio_001.wav
│   └── ...
└── gangwon/           # 강원도 방언 (선택사항)
    ├── audio_001.wav
    └── ...
```

### 2. 오디오 파일 요구사항
- 형식: WAV 또는 MP3
- 샘플링 레이트: 가급적 16kHz (자동으로 리샘플링됨)
- 길이: 2초 ~ 30초 권장
- 각 방언당 최소 100개 이상의 샘플 권장

### 3. 필요한 라이브러리
이 노트북 실행 전에 다음 명령어로 필요한 라이브러리를 설치하세요:
```bash
pip install transformers datasets torch torchaudio accelerate librosa soundfile scikit-learn evaluate jiwer
```


## Step 1: 환경 설정 및 라이브러리 임포트


In [ ]:
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List, Tuple

# Hugging Face libraries
from datasets import load_dataset, Audio, DatasetDict
from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2ForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from transformers import EarlyStoppingCallback

# Evaluation
import evaluate

# Audio processing
import librosa
import soundfile as sf

# ML utilities
from sklearn.metrics import classification_report, confusion_matrix
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")


## Step 2: 설정 (여기를 수정하세요!)


In [ ]:
# ============================================
# 🔧 여기를 본인의 환경에 맞게 수정하세요!
# ============================================

# 데이터셋 경로 (SSD에 저장된 방언 데이터 경로)
DATASET_PATH = "/path/to/your/ssd/dialect_dataset"  # 여기를 수정!

# 모델 저장 경로
OUTPUT_DIR = "../models/dialect_classifier"

# 사전 학습된 한국어 Wav2Vec2 모델
# 옵션 1: kresnik/wav2vec2-large-xlsr-korean (추천)
# 옵션 2: facebook/wav2vec2-large-xlsr-53 (다국어)
BASE_MODEL = "kresnik/wav2vec2-large-xlsr-korean"

# 학습 설정
BATCH_SIZE = 8  # GPU 메모리에 따라 조정 (8, 16, 32)
NUM_EPOCHS = 10
LEARNING_RATE = 3e-5
EVAL_STEPS = 100  # 평가 주기
SAVE_STEPS = 100  # 체크포인트 저장 주기

# 오디오 설정
SAMPLE_RATE = 16000
MAX_DURATION = 30  # 초 단위 (30초 이상은 잘림)

# GPU 사용 설정
USE_GPU = torch.cuda.is_available()
DEVICE = "cuda" if USE_GPU else "cpu"

print(f"📂 데이터셋 경로: {DATASET_PATH}")
print(f"💾 모델 저장 경로: {OUTPUT_DIR}")
print(f"🤖 사용할 베이스 모델: {BASE_MODEL}")
print(f"🖥️  디바이스: {DEVICE}")


## Step 3: 데이터셋 로드 및 전처리


In [ ]:
# AudioFolder를 사용하여 데이터셋 로드
print("📥 데이터셋을 로드하는 중...")

# 먼저 폴더가 존재하는지 확인
if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(
        f"데이터셋 경로가 존재하지 않습니다: {DATASET_PATH}\n"
        "위의 'Step 2: 설정' 셀에서 DATASET_PATH를 올바르게 설정해주세요!"
    )

# 하위 디렉토리(방언 레이블) 확인
dialect_dirs = [d for d in os.listdir(DATASET_PATH) 
                if os.path.isdir(os.path.join(DATASET_PATH, d)) and not d.startswith('.')]

if not dialect_dirs:
    raise ValueError(
        f"{DATASET_PATH}에 방언 폴더가 없습니다!\n"
        "예: standard/, gyeongsang/, jeolla/, chungcheong/ 등"
    )

print(f"✅ 감지된 방언 카테고리: {dialect_dirs}")

# AudioFolder로 데이터셋 로드
dataset = load_dataset("audiofolder", data_dir=DATASET_PATH)

print(f"\n✅ 데이터셋 로드 완료!")
print(dataset)


In [ ]:
# 데이터셋 분할 (train/validation/test)
print("\n🔀 데이터셋을 train/val/test로 분할하는 중...")

# train과 나머지로 분할 (80% train, 20% temp)
train_test = dataset["train"].train_test_split(test_size=0.2, seed=42)

# 나머지를 validation과 test로 분할 (각 10%씩)
val_test = train_test["test"].train_test_split(test_size=0.5, seed=42)

# 최종 데이터셋 구성
dataset_split = DatasetDict({
    "train": train_test["train"],
    "validation": val_test["train"],
    "test": val_test["test"]
})

print("\n✅ 분할 완료:")
print(dataset_split)

# 레이블 정보 추출
label_names = dataset_split["train"].features["label"].names
num_labels = len(label_names)

print(f"\n📊 방언 레이블 ({num_labels}개):")
for idx, label in enumerate(label_names):
    train_count = sum(1 for x in dataset_split["train"] if x["label"] == idx)
    val_count = sum(1 for x in dataset_split["validation"] if x["label"] == idx)
    test_count = sum(1 for x in dataset_split["test"] if x["label"] == idx)
    print(f"  {idx}. {label:15s} - Train: {train_count:3d}, Val: {val_count:3d}, Test: {test_count:3d}")


## Step 4: 모델 및 프로세서 로드


In [ ]:
print(f"\n🤖 모델과 프로세서를 로드하는 중: {BASE_MODEL}")

# Processor 로드 (오디오 전처리용)
processor = Wav2Vec2Processor.from_pretrained(BASE_MODEL)

# 모델 로드 (분류 헤드 추가)
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    BASE_MODEL,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_names)},
    id2label={i: label for i, label in enumerate(label_names)},
)

# 모델을 GPU로 이동 (가능한 경우)
model = model.to(DEVICE)

print(f"✅ 모델 로드 완료! (파라미터 수: {model.num_parameters():,})")
print(f"✅ 프로세서 로드 완료!")


## Step 5: 데이터 전처리 함수


In [ ]:
def preprocess_function(examples):
    """
    오디오 데이터를 모델 입력 형태로 변환
    """
    # 오디오를 16kHz로 리샘플링
    audio_arrays = [x["array"] for x in examples["audio"]]
    
    # 프로세서로 변환 (자동으로 패딩 및 정규화)
    inputs = processor(
        audio_arrays,
        sampling_rate=processor.feature_extractor.sampling_rate,
        return_tensors="pt",
        padding=True,
        max_length=int(processor.feature_extractor.sampling_rate * MAX_DURATION),
        truncation=True,
    )
    
    return {
        "input_values": inputs.input_values,
        "labels": examples["label"]
    }

print("⚙️ 데이터 전처리 중...")
print("이 작업은 시간이 걸릴 수 있습니다 (수 분 ~ 수십 분)...")

# 오디오를 16kHz로 리샘플링
dataset_split = dataset_split.cast_column("audio", Audio(sampling_rate=SAMPLE_RATE))

# 전처리 적용 (배치 처리)
encoded_dataset = dataset_split.map(
    preprocess_function,
    batched=True,
    batch_size=8,
    remove_columns=dataset_split["train"].column_names,
)

print("\n✅ 데이터 전처리 완료!")
print(encoded_dataset)


## Step 6: 평가 메트릭 정의


In [ ]:
# Accuracy 메트릭 로드
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """
    학습 중 평가 메트릭 계산
    """
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

print("✅ 평가 메트릭 준비 완료!")


## Step 7: 학습 설정


In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=2,  # 메모리가 부족하면 이 값을 높이세요
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,
    save_steps=SAVE_STEPS,
    save_total_limit=3,  # 최대 3개의 체크포인트만 유지
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    warmup_ratio=0.1,
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    push_to_hub=False,
    fp16=USE_GPU,  # GPU 사용 시 mixed precision 활성화
    dataloader_num_workers=4,
    remove_unused_columns=False,
)

print("\n📋 학습 설정:")
print(f"  • Batch size: {BATCH_SIZE}")
print(f"  • Epochs: {NUM_EPOCHS}")
print(f"  • Learning rate: {LEARNING_RATE}")
print(f"  • Mixed precision (FP16): {USE_GPU}")
print(f"  • Output directory: {OUTPUT_DIR}")


## Step 8: Trainer 초기화 및 학습 시작

**주의**: 다음 셀은 실행 시간이 오래 걸립니다 (GPU 사용 시 1-3시간, CPU 사용 시 수 시간~수일)


In [ ]:
# Early Stopping 설정 (validation 성능이 개선되지 않으면 조기 종료)
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,  # 3번 연속 개선 없으면 종료
    early_stopping_threshold=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[early_stopping],
)

print("\n✅ Trainer 초기화 완료!")
print("\n🚀 학습을 시작합니다...\n")
print("=" * 80)

# 학습 시작
train_result = trainer.train()

print("\n" + "=" * 80)
print("\n✅ 학습 완료!\n")
print("학습 결과:")
print(train_result)


## Step 9: 모델 저장


In [ ]:
print("\n💾 최종 모델 저장 중...")

# 최종 모델 저장
final_model_path = f"{OUTPUT_DIR}/final_model"
trainer.save_model(final_model_path)
processor.save_pretrained(final_model_path)

print(f"\n✅ 모델이 저장되었습니다: {final_model_path}")
print("\n저장된 파일:")
for file in os.listdir(final_model_path):
    print(f"  • {file}")


## Step 10: 테스트 세트 평가 및 결과 분석


In [ ]:
print("\n📊 테스트 세트 평가 중...\n")

# 테스트 세트 평가
test_results = trainer.evaluate(encoded_dataset["test"])

print("\n" + "=" * 80)
print("테스트 결과:")
print("=" * 80)
for key, value in test_results.items():
    print(f"  {key}: {value:.4f}")

# 테스트 세트 예측
predictions = trainer.predict(encoded_dataset["test"])
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

# 상세 분류 리포트
print("\n" + "=" * 80)
print("상세 분류 리포트:")
print("=" * 80)
print(classification_report(
    true_labels, 
    predicted_labels, 
    target_names=label_names,
    digits=4
))

# Confusion Matrix
print("\n" + "=" * 80)
print("Confusion Matrix:")
print("=" * 80)
cm = confusion_matrix(true_labels, predicted_labels)
cm_df = pd.DataFrame(cm, index=label_names, columns=label_names)
print(cm_df)


## Step 11: 모델 테스트 (실제 오디오 파일로 추론)


In [ ]:
from transformers import pipeline
import tempfile

print("\n🧪 추론 파이프라인 생성 중...")

# 추론 파이프라인 생성
classifier = pipeline(
    "audio-classification",
    model=final_model_path,
    device=0 if USE_GPU else -1
)

print("✅ 파이프라인 준비 완료!\n")

# 테스트 샘플로 추론 (테스트 세트의 첫 5개)
print("=" * 80)
print("샘플 추론 테스트:")
print("=" * 80)

test_samples = dataset_split["test"].select(range(min(5, len(dataset_split["test"]))))

for idx, sample in enumerate(test_samples):
    audio_array = sample["audio"]["array"]
    true_label = label_names[sample["label"]]
    
    # 임시 파일로 저장 (pipeline은 파일 경로 또는 array를 받음)
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_file:
        sf.write(temp_file.name, audio_array, SAMPLE_RATE)
        temp_path = temp_file.name
    
    # 추론
    result = classifier(temp_path, top_k=len(label_names))
    
    # 결과 출력
    print(f"\n샘플 {idx + 1}:")
    print(f"  실제 레이블: {true_label}")
    print(f"  예측 결과:")
    for pred in result:
        print(f"    - {pred['label']:15s}: {pred['score']:.4f}")
    
    # 임시 파일 삭제
    os.remove(temp_path)

print("\n" + "=" * 80)


## 🎉 완료! 다음 단계

### ✅ 완료된 작업
1. 데이터셋 로드 및 전처리
2. Wav2Vec2 모델 파인튜닝
3. 모델 평가 및 저장
4. 추론 테스트

### 📦 생성된 파일
- 모델 파일: `../models/dialect_classifier/final_model/`
- 체크포인트: `../models/dialect_classifier/checkpoint-*/`
- 로그: `../models/dialect_classifier/logs/`

### 🚀 다음 단계

**HabitLink 애플리케이션에서 방언 분석 사용하기:**

1. 모델이 자동으로 `src/dialect_analyzer.py`에 통합되었습니다
2. 메인 애플리케이션 실행: `python main.py`
3. 분석 모듈 선택 시 "방언 분석"을 선택하면 됩니다!

**모델 개선 (선택사항):**
- 더 많은 데이터 수집
- Hyperparameter 튜닝 (learning rate, batch size 등)
- Data augmentation 적용 (노이즈 추가, 속도 변경 등)

**프로그래밍 방식으로 사용:**
```python
from src.dialect_analyzer import DialectAnalyzer

analyzer = DialectAnalyzer("../models/dialect_classifier/final_model")
result = analyzer.analyze("path/to/audio.wav")
print(result)  # {'Gyeongsang': 0.85, 'Seoul': 0.10, ...}
```


In [ ]:
print("\n" + "="*80)
print("🎉 모든 작업이 완료되었습니다!")
print("="*80)
print(f"\n모델 경로: {final_model_path}")
print("\n이제 HabitLink 메인 애플리케이션에서 방언 분석을 사용할 수 있습니다!")
print("\n다음 명령어로 테스트하세요:")
print("  cd .. && python main.py")
